In [17]:

// 1) What is the percentage of loss or gain based on previous month sales
// 2) What is the percentage of sales each month based on last 6 months sales ?


In [29]:

// 1) What is the percentage of loss or gain based on previous month sales

import org.apache.spark.sql.expressions._

val product_data = Seq(
    (1,"iphone","01-01-2023",1500000),
    (2,"samsung","01-01-2023",1100000),
    (3,"oneplus","01-01-2023",1100000),
    (1,"iphone","01-02-2023",1300000),
    (2,"samsung","01-02-2023",1120000),
    (3,"oneplus","01-02-2023",1120000),
    (1,"iphone","01-03-2023",1600000),
    (2,"samsung","01-03-2023",1080000),
    (3,"oneplus","01-03-2023",1160000),
    (1,"iphone","01-04-2023",1700000),
    (2,"samsung","01-04-2023",1800000),
    (3,"oneplus","01-04-2023",1170000),
    (1,"iphone","01-05-2023",1200000),
    (2,"samsung","01-05-2023",980000),
    (3,"oneplus","01-05-2023",1175000),
    (1,"iphone","01-06-2023",1100000),
    (2,"samsung","01-06-2023",1100000),
    (3,"oneplus","01-06-2023",1200000)
).toDF("id","brand","date","price")

val formatted_dates = product_data.withColumn("date",date_format(to_date($"date", "dd-MM-yyyy"), "yyyy-MM-dd"))

val previous_month_sales = formatted_dates.withColumn("previous_month_sale", lag($"price",1).over(Window.partitionBy($"brand").orderBy($"date")))
                                  
previous_month_sales.withColumn("percent_loss_gain", round(($"price" - $"previous_month_sale")*100/$"price",2)).show(false)                                            
                                                      

+---+-------+----------+-------+-------------------+-----------------+
|id |brand  |date      |price  |previous_month_sale|percent_loss_gain|
+---+-------+----------+-------+-------------------+-----------------+
|1  |iphone |2023-01-01|1500000|null               |null             |
|1  |iphone |2023-02-01|1300000|1500000            |-15.38           |
|1  |iphone |2023-03-01|1600000|1300000            |18.75            |
|1  |iphone |2023-04-01|1700000|1600000            |5.88             |
|1  |iphone |2023-05-01|1200000|1700000            |-41.67           |
|1  |iphone |2023-06-01|1100000|1200000            |-9.09            |
|3  |oneplus|2023-01-01|1100000|null               |null             |
|3  |oneplus|2023-02-01|1120000|1100000            |1.79             |
|3  |oneplus|2023-03-01|1160000|1120000            |3.45             |
|3  |oneplus|2023-04-01|1170000|1160000            |0.85             |
|3  |oneplus|2023-05-01|1175000|1170000            |0.43             |
|3  |o

import org.apache.spark.sql.expressions._
product_data: org.apache.spark.sql.DataFrame = [id: int, brand: string ... 2 more fields]
formatted_dates: org.apache.spark.sql.DataFrame = [id: int, brand: string ... 2 more fields]
previous_month_sales: org.apache.spark.sql.DataFrame = [id: int, brand: string ... 3 more fields]


In [33]:

// 2) What is the percentage of sales each month based on last 6 months sales ?

import org.apache.spark.sql.expressions._

val emp_data = Seq((1,"manish",50000,"IT","m"),
(2,"vikash",60000,"sales","m"),
(3,"raushan",70000,"marketing","m"),
(4,"mukesh",80000,"IT","m"),
(5,"priti",90000,"sales","f"),
(6,"nikita",45000,"marketing","f"),
(7,"ragini",55000,"marketing","f"),
(8,"rashi",100000,"IT","f"),
(9,"aditya",65000,"IT","m"),
(10,"rahul",50000,"marketing","m"),
(11,"rakhi",50000,"IT","f"),
(12,"akhilesh",90000,"sales","m")).toDF("id","name","price","dept","gender")

emp_data
  .withColumn("total_sales",sum($"price").over(Window.partitionBy($"dept").orderBy($"price")))
  .withColumn("percentage", round(($"total_sales" - $"price")*100/$"total_sales",2))
  .show(false)


+---+--------+------+---------+------+-----------+----------+
|id |name    |price |dept     |gender|total_sales|percentage|
+---+--------+------+---------+------+-----------+----------+
|1  |manish  |50000 |IT       |m     |100000     |50.0      |
|11 |rakhi   |50000 |IT       |f     |100000     |50.0      |
|9  |aditya  |65000 |IT       |m     |165000     |60.61     |
|4  |mukesh  |80000 |IT       |m     |245000     |67.35     |
|8  |rashi   |100000|IT       |f     |345000     |71.01     |
|6  |nikita  |45000 |marketing|f     |45000      |0.0       |
|10 |rahul   |50000 |marketing|m     |95000      |47.37     |
|7  |ragini  |55000 |marketing|f     |150000     |63.33     |
|3  |raushan |70000 |marketing|m     |220000     |68.18     |
|2  |vikash  |60000 |sales    |m     |60000      |0.0       |
|5  |priti   |90000 |sales    |f     |240000     |62.5      |
|12 |akhilesh|90000 |sales    |m     |240000     |62.5      |
+---+--------+------+---------+------+-----------+----------+



import org.apache.spark.sql.expressions._
emp_data: org.apache.spark.sql.DataFrame = [id: int, name: string ... 3 more fields]
